In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import catboost as catboost


df_earner = pd.read_csv("/content/uber_hackathon_v2_mock_data.xlsx - earners.csv")
df_rides = pd.read_csv("/content/uber_hackathon_v2_mock_data.xlsx - rides_trips.csv")
df_eats = pd.read_csv("/content/uber_hackathon_v2_mock_data.xlsx - eats_orders.csv")
df_weather = pd.read_csv("/content/uber_hackathon_v2_mock_data.xlsx - weather_daily.csv")
df_surge = pd.read_csv("/content/uber_hackathon_v2_mock_data.xlsx - surge_by_hour.csv")


df_surge["start_time"] = df_surge["hour"]
df_surge = df_surge.drop("hour", axis=1)

In [3]:
df_surge.head()

,city_id,surge_multiplier,start_time
0,1,0.97,0
1,1,0.98,1
2,1,1.06,2
3,1,1.14,3
4,1,0.96,4


In [4]:
df_weather.head()

,date,city_id,weather
0,2023-01-01,1,clear
1,2023-01-01,2,rain
2,2023-01-01,3,clear
3,2023-01-01,4,clear
4,2023-01-01,5,clear


In [5]:
df_eats.head()

,order_id,courier_id,customer_id,merchant_id,city_id,vehicle_type,is_ev,start_time,end_time,pickup_lat,...,drop_lon,drop_hex_id9,distance_km,duration_mins,basket_value_eur,delivery_fee_eur,tip_eur,net_earnings,payment_type,date
0,b9df6e21-db8c-4d99-8596-67d0b3ac1f7c,E10249,C3215,M107,1,scooter,True,2023-04-22 13:22:00,2023-04-22 13:53:00,52.41501,...,4.90300,899a29fd4706501,3.52,31,20.69,3.81,0.00,3.81,card,2023-04-22
1,fddeb5a4-57db-4166-b279-1b189371c6cf,E10225,C3324,M506,5,bike,True,2023-04-30 05:57:00,2023-04-30 06:29:00,52.09705,...,4.28517,89866d623bdf588,5.09,32,20.64,3.88,0.00,3.88,wallet,2023-04-30
2,2fc3861e-4a4b-421f-abec-5ecd5fd94b67,E10162,C3369,M406,4,bike,True,2023-01-25 21:39:00,2023-01-25 21:49:00,51.45805,...,5.44989,8905254a561cb08,1.38,10,26.44,4.02,0.26,4.28,wallet,2023-01-25
3,7f7cc1ef-625c-4d0b-ba56-43bb2378256b,E10347,C3651,M506,5,scooter,True,2023-04-19 22:18:00,2023-04-19 22:48:00,52.09622,...,4.29106,896e5849fc58d1a,2.82,30,28.56,4.39,0.00,4.39,card,2023-04-19
4,f3c67354-8546-4fbe-970e-383c848361c5,E10269,C3449,M202,2,bike,True,2023-01-04 12:07:00,2023-01-04 12:23:00,51.93573,...,4.49094,89dd9b26161bfb1,4.72,16,29.78,3.58,0.00,3.58,wallet,2023-01-04


In [6]:
df_rides.head()

,ride_id,driver_id,rider_id,city_id,product,vehicle_type,is_ev,start_time,end_time,pickup_lat,...,drop_hex_id9,distance_km,duration_mins,surge_multiplier,fare_amount,uber_fee,net_earnings,tips,payment_type,date
0,58e20c15-9b2e-4458-ac00-107bcf9564cd,E10111,R2231,3,UberX,car,False,2023-01-13 23:50:00,2023-01-14 00:10:00,52.09319,...,896326c3e14b5c2,9.00,20,1.22,17.54,4.38,13.16,0.00,cash,2023-01-13
1,e02cff2b-bb4f-4b76-a97a-a2cca9477275,E10152,R2133,4,UberX,car,False,2023-02-10 05:13:00,2023-02-10 05:25:00,51.41972,...,898e2394712bd2c,4.79,12,1.01,8.48,2.12,6.36,1.80,card,2023-02-10
2,779d0691-f33d-4d00-991b-983d3171e420,E10157,R2534,3,UberPool,car,True,2023-02-25 18:27:00,2023-02-25 18:53:00,52.05469,...,8947d4823562561,6.59,26,1.36,12.87,3.22,9.65,0.00,card,2023-02-25
3,b08070cf-4a5f-432f-a04d-4fa35382aa87,E10024,R2265,2,UberGreen,car,True,2023-02-20 23:58:00,2023-02-21 00:24:00,51.94535,...,89cf8728790df33,8.24,26,1.22,17.96,4.49,13.47,0.51,wallet,2023-02-20
4,c5479c2b-d878-4986-bc0c-112284e8c495,E10146,R2126,3,UberX,car,True,2023-01-16 18:42:00,2023-01-16 19:13:00,52.10560,...,8900b3581cd6ec3,7.50,31,1.36,17.15,4.29,12.86,0.00,wallet,2023-01-16


In [7]:
df_earner.head()

,earner_id,earner_type,vehicle_type,fuel_type,is_ev,experience_months,rating,status,home_city_id
0,E10000,driver,car,hybrid,False,72,4.79,offline,4
1,E10001,driver,car,hybrid,False,76,4.77,online,2
2,E10002,driver,car,gas,False,61,4.45,engaged,5
3,E10003,driver,car,hybrid,False,13,4.54,offline,3
4,E10004,driver,car,EV,True,69,4.86,engaged,1


In [8]:
def HexaToInt(hex_str):
  hex_str = int(hex_str, 16)
  #print(date)

  return hex_str

print(HexaToInt("89b5443252677be"))

620182021056395198


In [9]:
def SplitTimeGetHour(date):
  #print(date)
  date = date.split(" ")[1]
  date = date.split(":")[0]

  return date

print(SplitTimeGetHour("2023-01-13 23:50:00"))

23


In [10]:
def TransformDFRides(df_rides, df_earner):
  df_rides = df_rides.merge(df_weather,on=['city_id', 'date'],how='left')

  df_rides["earner_id"] = df_rides["driver_id"]
  df_rides = df_rides.merge(df_earner[["earner_id","earner_type","fuel_type","experience_months","rating"]], on="earner_id",how="left")


  df_rides["start_time"] = df_rides["start_time"].map(lambda x : SplitTimeGetHour(x))
  df_rides["total_earnings"] = df_rides["net_earnings"]+df_rides["tips"]
  df_rides["pickup_hex_id9"] = df_rides["pickup_hex_id9"].map(lambda x : HexaToInt(x))
  df_rides["drop_hex_id9"] = df_rides["drop_hex_id9"].map(lambda x : HexaToInt(x))


  df_rides.drop(["drop_hex_id9","pickup_hex_id9","fare_amount","uber_fee","tips","net_earnings","earner_id","ride_id","driver_id","rider_id","end_time","date"],axis=1,inplace=True)


  return df_rides

df_comb = TransformDFRides(df_rides.copy(), df_earner)
print(TransformDFRides(df_rides.copy(), df_earner.copy()).dtypes)


city_id                int64
product               object
vehicle_type          object
is_ev                   bool
start_time            object
pickup_lat           float64
pickup_lon           float64
drop_lat             float64
drop_lon             float64
distance_km          float64
duration_mins          int64
surge_multiplier     float64
payment_type          object
weather               object
earner_type           object
fuel_type             object
experience_months      int64
rating               float64
total_earnings       float64
dtype: object


In [11]:
def TransformDFEats(df_eats, df_earner):
  df_eats = df_eats.merge(df_weather,on=['city_id', 'date'],how='left')



  df_eats["earner_id"] = df_eats["courier_id"]
  df_eats = df_eats.merge(df_earner[["earner_id","earner_type","fuel_type","experience_months","rating"]], on="earner_id",how="left")

  df_eats["start_time"] = df_eats["start_time"].map(lambda x : SplitTimeGetHour(x))

  df_eats["start_time"] = df_eats["start_time"].astype(int)
  df_eats = df_eats.merge(df_surge,on=['city_id', 'start_time'],how='left')

  #df_eats["surge_multiplier"] = 1

  df_eats["total_earnings"] = df_eats["net_earnings"]+df_eats["tip_eur"]

  df_eats["pickup_hex_id9"] = df_eats["pickup_hex_id9"].map(lambda x : HexaToInt(x))
  df_eats["drop_hex_id9"] = df_eats["drop_hex_id9"].map(lambda x : HexaToInt(x))

  df_eats.drop(["drop_hex_id9","pickup_hex_id9","tip_eur","net_earnings","delivery_fee_eur","merchant_id","basket_value_eur","earner_id","customer_id","order_id","courier_id","end_time","date"],axis=1,inplace=True)

  df_eats["product"] = "Uber Eats"

  return df_eats

df_comb = pd.concat([df_comb,TransformDFEats(df_eats.copy(), df_earner.copy())],axis=0, ignore_index=True)
print(TransformDFEats(df_eats.copy(), df_earner.copy()).dtypes)


city_id                int64
vehicle_type          object
is_ev                   bool
start_time             int64
pickup_lat           float64
pickup_lon           float64
drop_lat             float64
drop_lon             float64
distance_km          float64
duration_mins          int64
payment_type          object
weather               object
earner_type           object
fuel_type             object
experience_months      int64
rating               float64
surge_multiplier     float64
total_earnings       float64
product               object
dtype: object


In [12]:
def TransformObjectToIndexPlusRepairObjects(df_comb):

  mapProduct = {'UberX':0, 'UberPool':1, 'UberGreen':2, 'UberBlack':3, 'Uber Eats':4}
  mapVehicleType = {'car':0, 'scooter':1, 'bike':2}
  mapPaymentType = {'cash':0, 'card':1, 'wallet':2}
  mapEarnerType = {'driver':0, 'courier':1}
  mapFuelType = {'hybrid':0, 'EV':1, 'gas':2}
  mapWeatherType = {'clear':0, 'rain':1, 'snow':2}


  df_comb["product"] = df_comb["product"].map(mapProduct)
  df_comb["vehicle_type"] = df_comb["vehicle_type"].map(mapVehicleType)
  df_comb["payment_type"] = df_comb["payment_type"].map(mapPaymentType)
  df_comb["earner_type"] = df_comb["earner_type"].map(mapEarnerType)
  df_comb["fuel_type"] = df_comb["fuel_type"].map(mapFuelType)
  df_comb["weather"] = df_comb["weather"].map(mapWeatherType)

  df_comb["start_time"] = df_comb["start_time"].astype(int)

  #df_comb["dif_lat"] = df_comb["drop_lat"]-df_comb["pickup_lat"]
  #df_comb["dif_lon"] = df_comb["drop_lon"]-df_comb["pickup_lon"]

  return df_comb

df_comb_new = TransformObjectToIndexPlusRepairObjects(df_comb.copy())
print(TransformObjectToIndexPlusRepairObjects(df_comb.copy()).dtypes)

city_id                int64
product                int64
vehicle_type           int64
is_ev                   bool
start_time             int64
pickup_lat           float64
pickup_lon           float64
drop_lat             float64
drop_lon             float64
distance_km          float64
duration_mins          int64
surge_multiplier     float64
payment_type           int64
weather                int64
earner_type            int64
fuel_type              int64
experience_months      int64
rating               float64
total_earnings       float64
dtype: object


In [13]:
df_comb_new.describe()

,city_id,product,vehicle_type,start_time,pickup_lat,pickup_lon,drop_lat,drop_lon,distance_km,duration_mins,surge_multiplier,payment_type,weather,earner_type,fuel_type,experience_months,rating,total_earnings
count,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.00000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000,5500.000000
mean,3.036545,2.214364,0.625091,11.619091,51.976523,4.843137,51.969486,4.845435,4.959062,25.881091,1.154396,1.18200,0.318364,0.454545,0.995636,41.310727,4.723987,6.768325
std,1.411746,1.776279,0.820378,6.896805,0.305696,0.427245,0.304500,0.430670,2.351066,11.207091,0.143495,0.74191,0.505929,0.497975,0.716609,24.220054,0.171114,4.170014
min,1.000000,0.000000,0.000000,0.000000,51.391290,4.239230,51.403720,4.243270,0.800000,7.000000,0.950000,0.00000,0.000000,0.000000,0.000000,1.000000,4.220000,1.740000
25%,2.000000,0.000000,0.000000,6.000000,51.906395,4.435462,51.897838,4.435083,3.260000,17.000000,1.030000,1.00000,0.000000,0.000000,0.000000,21.000000,4.610000,3.427500
50%,3.000000,2.000000,0.000000,12.000000,52.055850,4.904720,52.053560,4.900270,4.560000,25.000000,1.130000,1.00000,0.000000,0.000000,1.000000,41.000000,4.730000,5.395000
75%,4.000000,4.000000,1.000000,18.000000,52.110233,5.149082,52.105353,5.162535,6.452500,34.000000,1.280000,2.00000,1.000000,1.000000,2.000000,62.000000,4.860000,9.560000
max,5.000000,4.000000,2.000000,23.000000,52.415680,5.538430,52.405940,5.523290,14.370000,55.000000,1.450000,2.00000,2.000000,1.000000,2.000000,84.000000,5.000000,32.760000


In [14]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


X = df_comb_new.drop(["distance_km","duration_mins","total_earnings"],axis=1)


#cat_features = ["product", "vehicle_type", "payment_type", "earner_type", "fuel_type", "weather"]

#--------------------------------------DIST MODEL----------------------------------------------------

modelDist = CatBoostRegressor(iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=5,
    one_hot_max_size=10,
    random_seed=42,
    verbose=False
)
y = df_comb_new["distance_km"]

modelDist.fit(X,y)

y_pred = modelDist.predict(X)

print(mean_absolute_error(y, y_pred))

#--------------------------------------TIME MODEL----------------------------------------------------

modelTime = CatBoostRegressor(iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=5,
    one_hot_max_size=10,
    random_seed=42,
    verbose=False
)
y = df_comb_new["duration_mins"]

modelTime.fit(X,y)

y_pred = modelTime.predict(X)

print(mean_absolute_error(y, y_pred))

#--------------------------------------EARN MODEL----------------------------------------------------

modelEarn = CatBoostRegressor(iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=5,
    one_hot_max_size=10,
    random_seed=42,
    verbose=False
)
y = df_comb_new["total_earnings"]

modelEarn.fit(X,y)

y_pred = modelEarn.predict(X)

print(mean_absolute_error(y, y_pred))

#--------------------------------------LOC TO LOC MODELS----------------------------------------------------

#--------------------------------------LOC LOC CITY MODEL----------------------------------------------------

X = df_comb_new[["pickup_lat","pickup_lon"]]

modelLocLocCity = CatBoostRegressor(iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=5,
    one_hot_max_size=10,
    random_seed=42,
    verbose=False
)
y = df_comb_new["city_id"]

modelLocLocCity.fit(X,y)

y_pred = modelLocLocCity.predict(X)

print(mean_absolute_error(y, y_pred))

1.1310418713424157
6.686202820389785
1.4538537249731585
0.0010798813519620858


In [15]:
modelDist.save_model("catboost_modelDist.cbm")
modelTime.save_model("catboost_modelTime.cbm")
modelEarn.save_model("catboost_modelEarn.cbm")
modelLocLocCity.save_model("catboost_modelLocLocCity.cbm")

In [16]:
df_comb_new.drop(["distance_km","duration_mins","total_earnings"],axis=1).columns

Index(['city_id', 'product', 'vehicle_type', 'is_ev', 'start_time',
       'pickup_lat', 'pickup_lon', 'drop_lat', 'drop_lon', 'surge_multiplier',
       'payment_type', 'weather', 'earner_type', 'fuel_type',
       'experience_months', 'rating'],
      dtype='object')

In [29]:
df_comb_new[["earner_type","city_id","product","start_time","pickup_lat","pickup_lon","drop_lat","drop_lon","surge_multiplier","payment_type"]].to_csv("data_graph.csv",index=False)

In [17]:
#'''
from google.colab import files
files.download("catboost_modelDist.cbm")
files.download("catboost_modelTime.cbm")
files.download("catboost_modelEarn.cbm")
files.download("catboost_modelLocLocCity.cbm")
#'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import time

df_aux = df_comb_new.copy()
df_aux["moneyPerHour"] = df_aux["total_earnings"]/(df_aux["duration_mins"]/60)

for i in range(0,2):
  for j in range(0,24):
    df_HeatMap = df_aux[df_aux["earner_type"]==i]
    df_HeatMap = df_HeatMap[df_HeatMap["start_time"]==j]

    #print(df_HeatMap)
    #print(df_HeatMap.head())
    df_HeatMap[["pickup_lat","pickup_lon","moneyPerHour"]].to_csv("HeatMap-"+str(i)+"-"+str(j)+".csv",index=False)
    #files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
    #time.sleep(2)

In [19]:
'''
for i in range(0,1):
  for j in range(0,10):
    files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
'''
for i in range(0,1):
  for j in range(10,20):
    files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
'''
for i in range(0,1):
  for j in range(20,24):
    files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
'''
for i in range(1,2):
  for j in range(0,10):
    files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
'''
for i in range(1,2):
  for j in range(10,20):
    files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
'''
for i in range(1,2):
  for j in range(20,24):
    files.download("HeatMap-"+str(i)+"-"+str(j)+".csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
'''
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Example DataFrame
# Scatter plot with color based on category

df_aux = df_comb_new

df_aux["special"] = df_aux["total_earnings"]/df_aux["duration_mins"]

plt.figure(figsize=(8,8))
sns.scatterplot(data=df_aux, x='rating', y='experience_months', hue='total_earnings', palette='viridis', s=50)
'''

'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Example DataFrame\n# Scatter plot with color based on category\n\ndf_aux = df_comb_new\n\ndf_aux["special"] = df_aux["total_earnings"]/df_aux["duration_mins"]\n\nplt.figure(figsize=(8,8))\nsns.scatterplot(data=df_aux, x=\'rating\', y=\'experience_months\', hue=\'total_earnings\', palette=\'viridis\', s=50)\n'